In [ ]:
#Collects data and a timestamp every 2 seconds and adds it to testdata.csv;
import RPi.GPIO as io
import os
import time
import datetime
from gpiozero import Robot, DigitalInputDevice
from time import sleep

def binarytotemp(binary, positive): # binary is a string, positive is a boolean
   if positive:
       return sum([2**i for i in range(12) if binary[11-i]=="1"])/16
   else:
       return -(sum([2**i for i in range(12) if binary[11-i]=="0"])+1)/16
   

#writes the current PID to the PID.txt file for access by remote connection
#the PID number is necessary for stopping the python file from remote connection
os.chdir("/")

TimeToChange = 5
with open("home/pi/Desktop/pid.txt", 'w') as PID:
   PID.write(str(os.getpid()))
#sets the pin numbering mode i think to board mode or something
io.setmode(io.BCM)
#sets up pin "24" as an io.in pin
io.setup(24, io.IN)

temperature=0;
alltext="";
counter=-1

#loop
while True: #infinite loop
   ###it makes a timestamp for the data that is being collected
   #appends the entry to testdata.csv
   with open("sys/bus/w1/devices/28-000008fe746d/w1_slave", 'r') as source:
       alltext=source.read()
       while alltext[counter]!="=":
           counter=counter-1;
       temperature=alltext[counter+1:]/1000
       source.close()
       
   with open("home/pi/Desktop/data.csv", 'a+') as data:
   #get date technique
       dateNow=str(datetime.datetime.now().strftime("%m/%d"))
       timeNow=str(int(datetime.datetime.now().strftime("%H"))*60+int(datetime.datetime.now().strftime("%M")))
   #writes comma separated date or time stamp and the ***input from the pin***

       entry = dateNow + "," + timeNow + "," + str(temperature) + "\n"
       data.write(entry)
       data.close()
   #prints to console for easy debugging
   print(entry)
   #gap between each collection cycle
   time.sleep(TimeToChange)

io.cleanup()